# Proyecto de visualización

## Introducción

El sector residencial desempeña un papel clave en el consumo energético total y en las emisiones asociadas al cambio climático. En países con climas fríos como Suiza, la **calefacción de los hogares** representa una parte sustancial del consumo final de energía y está fuertemente influenciada tanto por las **condiciones meteorológicas** como por la **tecnología de los sistemas de calefacción** utilizados.

El objetivo de este proyecto es analizar y visualizar la evolución del consumo energético residencial en Suiza entre los años **2000 y 2024**, poniendo especial énfasis en tres dimensiones interrelacionadas:

1. **Consumo energético por uso final**, con especial atención al peso de la calefacción frente a otros usos domésticos.
2. **Influencia del clima**, medida a través de indicadores como los *Heating Degree Days (HDD)*, y su impacto en la demanda de calefacción.
3. **Transición tecnológica de los sistemas de calefacción**, incluyendo la creciente adopción de bombas de calor y la reducción del uso de energías fósiles.

Para ello, se utilizan datos oficiales procedentes de:
- La **Oficina Federal de Energía (BFE)**, a través de las tablas PHH sobre consumo energético residencial.
- **MeteoSwiss**, para los datos climáticos diarios y los indicadores HDD/CDD.
- La **Oficina Federal de Estadística (BFS)**, para la información sobre sistemas de calefacción en edificios residenciales.

El trabajo se estructura en dos fases:
- **Preparación y limpieza de los datos**, realizada en este notebook mediante Python y pandas, transformando fuentes heterogéneas en conjuntos de datos *tidy* listos para visualización.
- **Visualización y análisis exploratorio**, llevado a cabo con Flourish, con el objetivo de construir una narrativa clara que conecte clima, consumo energético y cambio tecnológico.

Este enfoque permite no solo describir tendencias históricas, sino también aportar contexto explicativo sobre los factores que impulsan la evolución del consumo energético residencial en Suiza.

## Referencias:

https://stats.swiss/?lc=en

https://www.bfs.admin.ch/bfs/en/home/statistics/construction-housing/buildings/energy-sector.gnpdetail.2025-0428.html

https://www.bfs.admin.ch/bfs/en/home/statistics/construction-housing/surveys/gws2009.html

https://www.bfs.admin.ch/bfs/en/home/statistics/construction-housing/buildings/energy-sector.assetdetail.36162970.html

https://github.com/sdmx-twg/sdmx-rest/blob/v1.5.0/v2_1/ws/rest/docs/4_1_introduction.md

## Setup

In [ ]:
# ===============================
# Manipulación y análisis de datos
# ===============================
import pandas as pd
import numpy as np

# ===============================
# Trabajo con rutas y archivos
# ===============================
from pathlib import Path
from glob import glob
import os

# ===============================
# Expresiones regulares (limpieza de columnas, detección de años)
# ===============================
import re

# ===============================
# Descarga de datos desde APIs / web
# ===============================
import requests

# ===============================
# Control de tiempos (pausas en descargas)
# ===============================
import time

## Sistemas de calefacción en edificios residenciales

### Fuente de datos y alcance

La información sobre los **sistemas de calefacción en edificios residenciales** procede de la **Oficina Federal de Estadística de Suiza (BFS)**, concretamente de la estadística **GWS (Gebäude- und Wohnungsstatistik)**. Esta fuente recopila información detallada a nivel de edificio sobre el tipo de sistema de calefacción utilizado, permitiendo analizar su distribución territorial y su evolución temporal.

En este proyecto se utilizan los datos publicados a través de la plataforma **stats.swiss**, accesibles tanto mediante descarga directa como a través de la **API SDMX del BFS**. El análisis se centra en el período **2021–2024**, correspondiente a los últimos datos disponibles de forma homogénea a nivel cantonal.

Los sistemas de calefacción considerados incluyen, entre otros:
- Fuel (heating oil)
- Gas
- Electricidad
- Calefacción urbana (district heating)
- Bombas de calor
- Biomasa (madera)
- Solar térmica
- Otros sistemas y edificios sin sistema de calefacción declarado

### Proceso de preparación de los datos

El flujo de trabajo seguido para la preparación de los datos de calefacción es el siguiente:

1. **Descarga de los datos brutos**  
   Se parte de un archivo CSV obtenido desde stats.swiss que contiene el número de edificios clasificados por:
   - cantón,
   - año,
   - tipo de sistema de calefacción (codificado),
   - número de edificios.

2. **Limpieza y recodificación**  
   Los códigos numéricos asociados a los sistemas de calefacción se transforman en etiquetas descriptivas (por ejemplo, *Heat pump*, *Gas*, *Heating oil*), facilitando su interpretación y uso en las visualizaciones.

3. **Agregación y cálculo de indicadores**  
   Para cada combinación de **cantón y año**:
   - se calcula el número total de edificios,
   - se obtiene la cuota relativa (*share*) de cada sistema de calefacción,
   - se verifica la coherencia de los datos (la suma de las cuotas por cantón y año es igual a 1).

4. **Estructuración en formato tidy**  
   Los datos se transforman a un formato largo (*tidy data*), en el que cada fila representa:
   - un cantón,
   - un año,
   - un sistema de calefacción,
   - el número de edificios,
   - la cuota relativa del sistema.

### Archivos generados

Como resultado del proceso de preparación se generan los siguientes archivos:

- **`heating_systems_by_canton_year.csv`**  
  Contiene los datos agregados por cantón, año y sistema de calefacción, incluyendo el número absoluto de edificios.

- **`heating_systems_by_canton_year_tidy.csv`**  
  Versión final en formato *tidy*, que incorpora el número total de edificios por cantón y año, así como la cuota relativa de cada sistema de calefacción.

Estos archivos constituyen la base para analizar la **penetración de las bombas de calor**, la reducción de sistemas fósiles y las diferencias regionales entre cantones.

https://stats.swiss/vis?lc=en&df[ds]=disseminate&df[id]=DF_GWS_REG3&df[ag]=CH1.GWS&df[vs]=1.0.0&dq=A.8011%2B8012%2B8013%2B8014%2B8015%2B8016%2B8017%2B8018%2B8019%2B8020%2B8021%2B8022%2B8023%2B8024%2B_T.1021%2B1025%2B1030%2B1040.1%2B2%2B3%2B4%2B5%2B6%2B7%2B8%2B9.8100%2BZH%2BBE%2BLU%2BUR%2BSZ%2BOW%2BNW%2BGL%2BZG%2BFR%2BSO%2BBS%2BBL%2BSH%2BAR%2BAI%2BSG%2BGR%2BAG%2BTG%2BTI%2BVD%2BVS%2BNE%2BGE%2BJU&pd=2021%2C2024&to[TIME_PERIOD]=true&vw=tb

In [2]:
path = "data/heating_systems/raw/CH1.GWS_DF_GWS_REG3_2021_2024.csv"
df_raw = pd.read_csv(path)

df_raw.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,FREQ,Unnamed: 5,GBAUPS,Unnamed: 7,GKATS,Unnamed: 9,...,DIFF_LAST_UPDATE,Unnamed: 21,DIFF_EMBARGO_DATE,Unnamed: 23,DIFF_DB_STATE,Unnamed: 25,OBS_STATUS,Unnamed: 27,DIFF_REGION_REF,Unnamed: 29
0,DATAFLOW,CH1.GWS:DF_GWS_REG3(1.0.0),NaN,I,A,NaN,8020,NaN,1040,NaN,...,2025-09-22T08:30,NaN,2025-09-22T08:30,NaN,2025-09-22,NaN,A,NaN,POLG,NaN
1,DATAFLOW,CH1.GWS:DF_GWS_REG3(1.0.0),NaN,I,A,NaN,8020,NaN,1040,NaN,...,2025-09-22T08:30,NaN,2025-09-22T08:30,NaN,2025-09-22,NaN,A,NaN,POLG,NaN
2,DATAFLOW,CH1.GWS:DF_GWS_REG3(1.0.0),NaN,I,A,NaN,8020,NaN,1040,NaN,...,2025-09-22T08:30,NaN,2025-09-22T08:30,NaN,2025-09-22,NaN,A,NaN,POLG,NaN
3,DATAFLOW,CH1.GWS:DF_GWS_REG3(1.0.0),NaN,I,A,NaN,8020,NaN,1040,NaN,...,2025-09-22T08:30,NaN,2025-09-22T08:30,NaN,2025-09-22,NaN,A,NaN,POLG,NaN
4,DATAFLOW,CH1.GWS:DF_GWS_REG3(1.0.0),NaN,I,A,NaN,8022,NaN,1040,NaN,...,2025-09-22T08:30,NaN,2025-09-22T08:30,NaN,2025-09-22,NaN,A,NaN,POLG,NaN


In [3]:
IN_PATH = "data/heating_systems/heating_systems_by_canton_year.csv" 
df = pd.read_csv(IN_PATH)

# 1) Agregar: canton-year-heating_system (sumando OBS)
agg = (
    df.groupby(["canton", "year", "heating_system"], as_index=False)
      .agg(n_buildings=("n_buildings", "sum"))
)

# 2) Recalcular totales y shares
tot = (
    agg.groupby(["canton", "year"], as_index=False)["n_buildings"]
       .sum()
       .rename(columns={"n_buildings": "total_buildings"})
)

final = agg.merge(tot, on=["canton", "year"], how="left")
final["share"] = final["n_buildings"] / final["total_buildings"]

# 3) (Opcional) quitar el total Suiza si aparece como canton=8100
final = final[final["canton"] != 8100].copy()

# 4) Comprobación: share suma 1 por canton-year
check = final.groupby(["canton", "year"])["share"].sum().reset_index()
print("Max error vs 1:", (check["share"] - 1).abs().max())

OUT_PATH = "data/heating_systems/heating_systems_by_canton_year_tidy.csv"
final.to_csv(OUT_PATH, index=False)
print("Exportado:", OUT_PATH)

final.head(15)


Max error vs 1: 1.1102230246251565e-16
Exportado: data/heating_systems/heating_systems_by_canton_year_tidy.csv


,canton,year,heating_system,n_buildings,total_buildings,share
0,8100,2021,Biomass,127716,3548322,0.035993
1,8100,2021,District heating,419502,3548322,0.118225
2,8100,2021,Electricity,1445536,3548322,0.407386
3,8100,2021,Gas,624202,3548322,0.175915
4,8100,2021,Heat pump,285608,3548322,0.080491
5,8100,2021,Heating oil,604760,3548322,0.170435
6,8100,2021,No heating system,10404,3548322,0.002932
7,8100,2021,Other,18494,3548322,0.005212
8,8100,2021,Solar thermal,12100,3548322,0.003410
9,8100,2022,Biomass,134968,3570642,0.037799


In [4]:
final[["canton", "year"]].drop_duplicates().sort_values(["year", "canton"])

,canton,year
0,8100,2021
36,AG,2021
72,AI,2021
108,AR,2021
144,BE,2021
...,...,...
819,UR,2024
855,VD,2024
891,VS,2024
927,ZG,2024


In [5]:
final.groupby(["canton", "year"])["share"].sum().describe()

count    1.080000e+02
mean     1.000000e+00
std      2.839667e-17
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: share, dtype: float64

In [6]:
hp_2024 = (
    final[(final["year"] == 2024) & (final["heating_system"] == "Heat pump")]
    .sort_values("share", ascending=False)
)

hp_2024[["canton", "share"]].head(10)

,canton,share
895,VS,0.222358
787,TI,0.160230
391,GR,0.120455
823,UR,0.100571
859,VD,0.098595
571,OW,0.098136
355,GL,0.092190
535,NW,0.084302
283,FR,0.074681
31,8100,0.070768


In [7]:
hp_trend = final[final["heating_system"] == "Heat pump"]
hp_trend.groupby(["year"])["share"].mean().reset_index()

,year,share
0,2021,0.074672
1,2022,0.073217
2,2023,0.069199
3,2024,0.065533


In [8]:
heating_dict = {
    1: "Heating oil",
    2: "Gas",
    3: "Electricity",
    4: "District heating",
    5: "Heat pump",
    6: "Biomass",
    7: "Solar thermal",
    8: "Other",
    9: "No heating system"
}

In [9]:
######################################
# Agregación y porcentajes
######################################

df = df_raw[["GEMEINDENAME", "TIME_PERIOD", "GWAERZH", "OBS_VALUE"]].copy()

df.rename(columns={
    "GEMEINDENAME": "canton",
    "TIME_PERIOD": "year",
    "GWAERZH": "heating_code",
    "OBS_VALUE": "n_buildings"
}, inplace=True)

df["heating_system"] = df["heating_code"].map(heating_dict)


In [10]:
totals = df.groupby(["canton", "year"])["n_buildings"].sum().reset_index()
df = df.merge(totals, on=["canton", "year"], suffixes=("", "_total"))

df["share"] = df["n_buildings"] / df["n_buildings_total"]

In [11]:
#####################################
# Export final
#####################################

out_path = "data/heating_systems/heating_systems_by_canton_year.csv"
df.to_csv(out_path, index=False)


## Consumo energético residencial

### Fuente de datos y alcance

El análisis del **consumo energético residencial** se basa en las tablas **PHH (Private Haushalte)** publicadas por la **Oficina Federal de Energía de Suiza (BFE)**. Estas tablas ofrecen una reconstrucción coherente y homogénea del consumo energético de los hogares suizos desde el año **2000 hasta 2024**, expresada en **petajulios (PJ)**.

Los datos PHH se obtienen a partir de modelos energéticos que combinan estadísticas oficiales, encuestas sectoriales y supuestos técnicos, permitiendo descomponer el consumo energético residencial tanto por **uso final** como por **tipo de energía**. En este proyecto se utilizan varias tablas clave para capturar diferentes perspectivas del consumo energético doméstico.

### Dimensiones analizadas

El consumo energético residencial se estudia a través de las siguientes dimensiones complementarias:

1. **Consumo por uso final**  
   A partir de la *Tabelle 8*, se analiza cómo se reparte el consumo energético de los hogares entre distintos usos, como:
   - calefacción de espacios (*Raumwärme*),
   - agua caliente sanitaria,
   - cocina,
   - iluminación,
   - electrodomésticos y usos eléctricos específicos.

   Esta descomposición permite identificar qué usos concentran la mayor parte de la demanda energética y cómo han evolucionado en el tiempo.

2. **Consumo térmico por uso**  
   Mediante la *Tabelle 9*, se profundiza en el consumo de **energía térmica**, diferenciando entre calefacción de espacios, agua caliente y otros usos térmicos. Esta tabla resulta especialmente relevante para contextualizar el peso de la calefacción dentro del consumo energético total.

3. **Calefacción y condiciones meteorológicas**  
   Las *Tabellen 11 y 12* permiten analizar el consumo de calefacción por tipo de energía:
   - con influencia directa de la meteorología,
   - y ajustado por condiciones climáticas medias.

   Esta distinción es fundamental para separar los efectos estructurales (eficiencia, cambio tecnológico) de las variaciones debidas a inviernos más fríos o más suaves.

### Proceso de preparación de los datos

Para su uso en las visualizaciones, los datos PHH se someten a un proceso sistemático de preparación:

- limpieza de columnas no informativas,
- identificación y selección de columnas correspondientes a años,
- transformación a formato largo (*tidy data*),
- homogeneización de nombres y unidades.

El resultado son conjuntos de datos consistentes y comparables en el tiempo, adecuados para su integración con información climática y tecnológica.

### Archivos generados

Como resultado de este proceso, se generan los siguientes archivos CSV en formato *tidy*:

- **`tabelle8_residential_energy_by_use_2000_2024.csv`**  
  Consumo energético residencial por uso final.

- **`tabelle9_thermal_energy_by_use_2000_2024.csv`**  
  Consumo de energía térmica por uso.

- **`tabelle11_space_heating_by_energy_carrier_weather_influenced_2000_2024.csv`**  
  Consumo de calefacción por tipo de energía con influencia meteorológica.

- **`tabelle12_space_heating_by_energy_carrier_weather_adjusted_2000_2024.csv`**  
  Consumo de calefacción ajustado por condiciones climáticas.

Estos archivos constituyen la base para las visualizaciones desarrolladas en Flourish y permiten analizar de forma integrada la evolución del consumo energético residencial, su relación con el clima y los cambios tecnológicos en los sistemas de calefacción.

### Papel en la narrativa del proyecto

El análisis del consumo energético residencial proporciona el **marco general** del proyecto. Permite identificar tendencias de largo plazo, como la reducción del consumo térmico total o el aumento del peso relativo de los usos eléctricos, y sirve como punto de partida para relacionar estos cambios con la evolución climática y la transición de los sistemas de calefacción en los hogares suizos.


In [26]:
path = "data/energy/12388-PHH_Webtabellen_2024.xlsx"

xls = pd.ExcelFile(path)
xls.sheet_names

['Titelblatt',
 'Tabellenverzeichnis',
 'Tabelle1',
 'Tabelle2',
 'Tableau3',
 'Tableau4',
 'Tabelle6',
 'Tabelle7',
 'Tabelle8',
 'Tabelle9',
 'Tabelle10',
 'Tabelle11',
 'Tabelle12',
 'Tabelle13',
 'Tabelle14',
 'Tabelle15',
 'Tabelle16',
 'Tabelle17',
 'Tabelle18',
 'Tabelle19',
 'Tabelle20',
 'Tabelle21',
 'Tabelle22']

In [13]:
# Ruta y hoja
path = Path("data/energy/12388-PHH_Webtabellen_2024.xlsx")
sheet = "Tabelle8"

# 1) Leer usando la cabecera correcta
df_raw = pd.read_excel(path, sheet_name=sheet, header=5)

# 2) Quitar columnas vacías tipo "Unnamed"
df_raw = df_raw.loc[:, ~df_raw.columns.astype(str).str.startswith("Unnamed")]

# 3) Renombrar primera columna
df_raw = df_raw.rename(columns={df_raw.columns[0]: "end_use"})

# 4) Detectar columnas que son años (2000 o 2000.0)
year_cols = [
    c for c in df_raw.columns
    if re.fullmatch(r"\d{4}(\.0)?", str(c))
]

print("Años detectados:", year_cols)

# 5) Quedarnos solo con usos + años
df_use = df_raw[["end_use"] + year_cols]

# 6) Pasar a formato largo (tidy)
df_t8 = df_use.melt(
    id_vars="end_use",
    var_name="year",
    value_name="consumption_PJ"
)

# 7) Limpiar tipos
df_t8["year"] = pd.to_numeric(df_t8["year"], errors="coerce").astype(int)
df_t8 = df_t8.dropna(subset=["consumption_PJ"])

df_t8.head(10)

Años detectados: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


,end_use,year,consumption_PJ
0,Raumwärme,2000,169.699611
1,Raumwärme fest installiert,2000,168.184271
2,Heizen mobil,2000,1.515340
3,Warmwasser,2000,31.564857
4,"Klima, Lüftung, HT",2000,3.508979
5,Heizen Hilfsenergie,2000,2.259114
6,"Lüftung, Luftbefeuchtung",2000,0.742629
7,Klimatisierung,2000,0.050461
8,"Antennenverstärker, u.a.",2000,0.456774
9,"I&K, Unterhaltung",2000,5.666800


In [14]:
out_path = Path("data/energy/processed/tabelle8_residential_energy_by_use_2000_2024.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)

df_t8.to_csv(out_path, index=False)

print("CSV guardado en:", out_path)

CSV guardado en: data\energy\processed\tabelle8_residential_energy_by_use_2000_2024.csv


In [15]:
# Archivo y hoja
path = Path("data/energy/12388-PHH_Webtabellen_2024.xlsx")
sheet = "Tabelle9"   # 

# 1) Leer usando la cabecera correcta
df_raw = pd.read_excel(path, sheet_name=sheet, header=5)

# 2) Quitar columnas vacías tipo "Unnamed"
df_raw = df_raw.loc[:, ~df_raw.columns.astype(str).str.startswith("Unnamed")].copy()

# 3) Renombrar la primera columna como "thermal_use"
df_raw = df_raw.rename(columns={df_raw.columns[0]: "thermal_use"})

# 4) Detectar columnas-año (2000 o 2000.0)
year_cols = [c for c in df_raw.columns if re.fullmatch(r"\d{4}(\.0)?", str(c).strip())]
print("Años detectados:", year_cols)

# 5) uso térmico + años
df_use = df_raw[["thermal_use"] + year_cols].copy()

# 6) Convertir a formato largo (tidy)
df_t9 = df_use.melt(
    id_vars="thermal_use",
    var_name="year",
    value_name="consumption_PJ"
)

# 7) Limpiar tipos y filas vacías
df_t9["year"] = pd.to_numeric(df_t9["year"], errors="coerce").astype(int)
df_t9 = df_t9.dropna(subset=["consumption_PJ"])

# (Opcional) quitar fila Total
# df_t9 = df_t9[df_t9["thermal_use"].str.lower() != "total"]

df_t9.head(10)


Años detectados: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


,thermal_use,year,consumption_PJ
0,Raumwärme,2000,157.002772
1,Warmwasser,2000,24.323419
2,Prozesswärme / Kochen,2000,0.804732
3,Total,2000,182.130923
13,Raumwärme,2001,170.756598
14,Warmwasser,2001,24.162267
15,Prozesswärme / Kochen,2001,0.769259
16,Total,2001,195.688124
26,Raumwärme,2002,158.213134
27,Warmwasser,2002,24.077755


In [16]:
out_path = Path("data/energy/processed/tabelle9_thermal_energy_by_use_2000_2024.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)

df_t9.to_csv(out_path, index=False)

print("CSV guardado en:", out_path)
print("Filas:", len(df_t9))
print(df_t9.head(10))


CSV guardado en: data\energy\processed\tabelle9_thermal_energy_by_use_2000_2024.csv
Filas: 100
              thermal_use  year  consumption_PJ
0               Raumwärme  2000      157.002772
1              Warmwasser  2000       24.323419
2   Prozesswärme / Kochen  2000        0.804732
3                   Total  2000      182.130923
13              Raumwärme  2001      170.756598
14             Warmwasser  2001       24.162267
15  Prozesswärme / Kochen  2001        0.769259
16                  Total  2001      195.688124
26              Raumwärme  2002      158.213134
27             Warmwasser  2002       24.077755


In [17]:
# Archivo y hoja
path = Path("data/energy/12388-PHH_Webtabellen_2024.xlsx")
sheet = "Tabelle11"   # <-- importante

# 1) Leer con cabecera correcta
df_raw = pd.read_excel(path, sheet_name=sheet, header=5)

# 2) Quitar columnas vacías
df_raw = df_raw.loc[:, ~df_raw.columns.astype(str).str.startswith("Unnamed")].copy()

# 3) Renombrar primera columna
df_raw = df_raw.rename(columns={df_raw.columns[0]: "energy_carrier"})

# 4) Detectar columnas de años (2000 o 2000.0)
year_cols = [
    c for c in df_raw.columns
    if re.fullmatch(r"\d{4}(\.0)?", str(c).strip())
]

print("Años detectados:", year_cols)

# 5) Seleccionar columnas útiles
df_use = df_raw[["energy_carrier"] + year_cols].copy()

# 6) Formato largo (tidy)
df_t11 = df_use.melt(
    id_vars="energy_carrier",
    var_name="year",
    value_name="space_heating_PJ"
)

# 7) Limpieza
df_t11["year"] = pd.to_numeric(df_t11["year"], errors="coerce").astype(int)
df_t11 = df_t11.dropna(subset=["space_heating_PJ"])

# (Opcional) quitar fila Total
# df_t11 = df_t11[df_t11["energy_carrier"].str.lower() != "total"]

df_t11.head(10)


Años detectados: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


,energy_carrier,year,space_heating_PJ
0,Heizöl,2000,102.953358
1,Erdgas,2000,28.604506
2,Kohle,2000,0.440424
3,El. Widerstandsheizungen,2000,11.237376
4,Elektrische Wärmepumpen,2000,1.459464
5,Fernwärme,2000,4.311263
6,Holz,2000,18.033402
7,Umweltwärme,2000,2.619819
8,Solar,2000,0.040000
9,Total,2000,169.699611


In [18]:
out_path = Path("data/energy/processed/tabelle11_space_heating_by_energy_carrier_weather_influenced_2000_2024.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)

df_t11.to_csv(out_path, index=False)

print("CSV guardado en:", out_path)
print("Filas:", len(df_t11))


CSV guardado en: data\energy\processed\tabelle11_space_heating_by_energy_carrier_weather_influenced_2000_2024.csv
Filas: 300


In [19]:
# Archivo y hoja
path = Path("data/energy/12388-PHH_Webtabellen_2024.xlsx")
sheet = "Tabelle12"

# 1) Leer con cabecera correcta
df_raw = pd.read_excel(path, sheet_name=sheet, header=5)

# 2) Quitar columnas vacías tipo Unnamed
df_raw = df_raw.loc[:, ~df_raw.columns.astype(str).str.startswith("Unnamed")].copy()

# 3) Renombrar primera columna
df_raw = df_raw.rename(columns={df_raw.columns[0]: "energy_carrier"})

# 4) Detectar columnas-año (2000 o 2000.0)
year_cols = [
    c for c in df_raw.columns
    if re.fullmatch(r"\d{4}(\.0)?", str(c).strip())
]
print("Años detectados:", year_cols)

# 5) Selección y formato tidy
df_use = df_raw[["energy_carrier"] + year_cols].copy()

df_t12 = df_use.melt(
    id_vars="energy_carrier",
    var_name="year",
    value_name="space_heating_PJ_adjusted"
)

# 6) Limpieza
df_t12["year"] = pd.to_numeric(df_t12["year"], errors="coerce").astype(int)
df_t12 = df_t12.dropna(subset=["space_heating_PJ_adjusted"])

# (Opcional) quitar fila Total
# df_t12 = df_t12[df_t12["energy_carrier"].str.lower() != "total"]

df_t12.head(10)


Años detectados: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


,energy_carrier,year,space_heating_PJ_adjusted
0,Heizöl,2000,115.122077
1,Erdgas,2000,32.005270
2,Kohle,2000,0.491980
3,El. Widerstandsheizungen,2000,12.359871
4,Elektrische Wärmepumpen,2000,1.628031
5,Fernwärme,2000,4.828914
6,Holz,2000,20.106866
7,Umweltwärme,2000,2.922293
8,Solar,2000,0.040000
9,Total,2000,189.505301


In [20]:
out_path = Path("data/energy/processed/tabelle12_space_heating_by_energy_carrier_weather_adjusted_2000_2024.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)

df_t12.to_csv(out_path, index=False)

print("CSV guardado en:", out_path)
print("Filas:", len(df_t12))


CSV guardado en: data\energy\processed\tabelle12_space_heating_by_energy_carrier_weather_adjusted_2000_2024.csv
Filas: 300


In [21]:
print("Años:", df_t12["year"].min(), "-", df_t12["year"].max())
print("Carriers (ejemplo):", df_t12["energy_carrier"].unique()[:10])

Años: 2000 - 2024
Carriers (ejemplo): ['Heizöl' 'Erdgas' 'Kohle' 'El. Widerstandsheizungen'
 'Elektrische Wärmepumpen' 'Fernwärme' 'Holz' 'Umweltwärme' 'Solar'
 'Total']


## Clima (para HDD/CDD) — 

## Clima y demanda energética residencial

### Motivación

Las condiciones climáticas influyen de forma directa en la demanda energética de los hogares, especialmente en países con inviernos fríos como Suiza. Las variaciones interanuales de la temperatura afectan principalmente al **consumo de calefacción**, que representa el mayor componente del consumo energético residencial.

Para analizar esta relación de forma cuantitativa, se utilizan indicadores climáticos sintéticos que permiten resumir la severidad térmica de un año y compararla de manera homogénea en el tiempo y el espacio.

### Indicadores climáticos: HDD y CDD

En este proyecto se emplean dos indicadores ampliamente utilizados en el análisis energético:

- **Heating Degree Days (HDD)**: miden la demanda potencial de calefacción. Un valor elevado de HDD indica un año más frío y, por tanto, una mayor necesidad de calefacción.
- **Cooling Degree Days (CDD)**: miden la demanda potencial de refrigeración, relevante principalmente en contextos de climatización estival.

Dado el contexto climático suizo, el análisis se centra principalmente en los **HDD**, mientras que los **CDD** se incluyen como referencia para observar tendencias recientes asociadas al aumento de temperaturas.

### Fuente de datos y cobertura

Los datos climáticos proceden de **MeteoSwiss**, el servicio meteorológico nacional de Suiza. Se utilizan registros diarios de temperatura de una red de estaciones meteorológicas distribuidas por todo el territorio nacional.

A partir de estos registros se calculan los indicadores HDD y CDD:
- a nivel de **estación meteorológica**,
- y posteriormente agregados a nivel **cantonal**.

El período de análisis abarca los años **2000–2024**, lo que permite estudiar tanto la variabilidad interanual como posibles tendencias de medio plazo.

https://opendatadocs.meteoswiss.ch/

https://opendatadocs.meteoswiss.ch/a-data-groundbased/a1-automatic-weather-stations#data-download

### Proceso de preparación de los datos

El flujo de trabajo seguido para los datos climáticos incluye:

1. **Descarga de series diarias de temperatura** desde la plataforma de datos abiertos de MeteoSwiss.
2. **Cálculo de HDD y CDD** a partir de la temperatura media diaria, siguiendo definiciones estándar utilizadas en estudios energéticos.
3. **Agregación temporal y espacial**, obteniendo valores anuales por estación y por cantón.
4. **Estructuración en formato tidy**, facilitando su integración con los datos de consumo energético y sistemas de calefacción.

### Archivos generados

Como resultado de este proceso se generan los siguientes conjuntos de datos:

- **`smn_hdd_cdd_station_year_2000_2024.csv`**  
  Indicadores HDD y CDD anuales a nivel de estación meteorológica.

- **`smn_hdd_cdd_canton_year_2000_2024.csv`**  
  Indicadores HDD y CDD agregados a nivel cantonal.

Estos archivos permiten analizar la evolución del clima en Suiza y proporcionan el contexto necesario para interpretar los cambios observados en el consumo energético residencial y en la demanda de calefacción.

### Papel en la narrativa del proyecto

La inclusión de indicadores climáticos permite distinguir entre:
- variaciones del consumo energético debidas a **factores meteorológicos** (años más fríos o más suaves),
- y cambios de carácter **estructural**, asociados a mejoras de eficiencia energética o a la transición hacia nuevos sistemas de calefacción.

De este modo, el análisis climático constituye un elemento clave para comprender la relación entre clima, consumo energético y transformación tecnológica en el sector residencial suizo.

In [ ]:
# ===============================
# Rutas
# ===============================
META_DIR = Path("data/data_info")
STATIONS_META  = META_DIR / "ogd-smn_meta_stations.csv"
INVENTORY_META = META_DIR / "ogd-smn_meta_datainventory.csv"

# CSV diarios
RAW_DIR = Path("data/meteo/smn_daily_historical_by_station")

PROC_DIR = Path("data/meteo/processed")
PROC_DIR.mkdir(parents=True, exist_ok=True)

OUT_STATION = PROC_DIR / "smn_hdd_cdd_station_year_2000_2024.csv"
OUT_CANTON  = PROC_DIR / "smn_hdd_cdd_canton_year_2000_2024.csv"

# ===============================
# Parámetros HDD/CDD
# ===============================
BASE_URL = "https://data.geo.admin.ch/ch.meteoswiss.messwerte-tageswerte"
BASE_H = 18.0
BASE_C = 22.0

TEMP_COL = "tre200d0"
DATE_COL = "reference_timestamp"
STATION_COL = "station_abbr"

# ===============================
# Helpers
# ===============================
def read_meta_csv(path: Path) -> pd.DataFrame:
    """metadatos OGD vienen con ';' y cp1252."""
    return pd.read_csv(path, sep=";", encoding="cp1252")

def detect_canton_col(stations_meta: pd.DataFrame) -> str:
    for c in ["station_canton", "station_canton ", "canton", "KT", "kanton"]:
        if c in stations_meta.columns:
            return c
    raise KeyError(f"No encuentro columna cantón. Columnas: {stations_meta.columns.tolist()}")

def compute_hdd_cdd(tmean: pd.Series, base_h=BASE_H, base_c=BASE_C):
    t = pd.to_numeric(tmean, errors="coerce")
    hdd = np.maximum(0.0, base_h - t)
    cdd = np.maximum(0.0, t - base_c)
    return hdd, cdd

def download_csv(url: str, out_path: Path, sleep_s: float = 0.1, timeout_s: int = 60) -> str:
    """Descarga solo si no existe."""
    if out_path.exists():
        return "SKIP"
    r = requests.get(url, timeout=timeout_s)
    r.raise_for_status()
    out_path.write_bytes(r.content)
    time.sleep(sleep_s)
    return "OK"

def ensure_hdd_cdd(force_rebuild: bool = False, max_stations: int | None = None):
    """
    - Si existen OUT_STATION y OUT_CANTON y force_rebuild=False -> carga y devuelve.
    - Si faltan o force_rebuild=True -> usa CSVs locales; si falta alguno intenta descargarlo.
    """
    # 0) modo rápido
    if (not force_rebuild) and OUT_STATION.exists() and OUT_CANTON.exists():
        return pd.read_csv(OUT_STATION), pd.read_csv(OUT_CANTON)

    # 1) metadatos
    stations_meta = read_meta_csv(STATIONS_META)
    inv = read_meta_csv(INVENTORY_META)

    CANTON_COL = detect_canton_col(stations_meta)
    print("Columna cantón detectada:", CANTON_COL)

    # 2) estaciones con tre200d0
    stations_with_temp = inv.loc[inv["parameter_shortname"] == TEMP_COL, STATION_COL].dropna().unique()
    stations = stations_meta[stations_meta[STATION_COL].isin(stations_with_temp)].copy()

    if max_stations is not None:
        stations = stations.head(max_stations)

    print("Estaciones objetivo con tre200d0:", len(stations))
    print("RAW_DIR usado:", RAW_DIR)

    # 3) asegurar archivos (descargar solo faltantes)
    ok = skip = fail = 0
    for abbr in stations[STATION_COL]:
        out = RAW_DIR / f"smn_{abbr}_d_historical.csv"
        if out.exists():
            skip += 1
            continue

        url = f"{BASE_URL}/smn_{abbr}_d_historical.csv"
        try:
            status = download_csv(url, out)
            ok += (status == "OK")
        except Exception:
            fail += 1

    print(f"Descarga faltantes -> OK: {ok} | SKIP (ya existían): {skip} | FAIL: {fail}")
    if fail > 0:
        print("Fallos de descarga: seguiré con los archivos locales disponibles.")

    # 4) cálculo HDD/CDD station-year
    frames = []
    skipped_files = 0
    missing_files = 0

    for abbr in stations[STATION_COL]:
        p = RAW_DIR / f"smn_{abbr}_d_historical.csv"
        if not p.exists():
            missing_files += 1
            continue

        # CSV diarios
        try:
            df = pd.read_csv(p)
        except UnicodeDecodeError:
            df = pd.read_csv(p, encoding="cp1252", encoding_errors="ignore")

        required = {STATION_COL, DATE_COL, TEMP_COL}
        if not required.issubset(df.columns):
            skipped_files += 1
            continue

        df["date"] = pd.to_datetime(df[DATE_COL], errors="coerce")
        df = df.dropna(subset=["date", TEMP_COL])
        df["year"] = df["date"].dt.year

        hdd, cdd = compute_hdd_cdd(df[TEMP_COL])
        df["HDD"] = hdd
        df["CDD"] = cdd

        frames.append(df.groupby([STATION_COL, "year"], as_index=False)[["HDD", "CDD"]].sum())

    if not frames:
        n_csv = len(list(RAW_DIR.glob("smn_*_d_historical.csv")))
        raise RuntimeError(
            f"No hay ficheros válidos en {RAW_DIR}.\n"
            f"CSVs encontrados: {n_csv}\n"
            f"Comprueba columnas: {STATION_COL}, {DATE_COL}, {TEMP_COL}."
        )

    df_station = pd.concat(frames, ignore_index=True)

    # añadir cantón y normalizar a 'canton'
    df_station = df_station.merge(
        stations[[STATION_COL, CANTON_COL]],
        on=STATION_COL,
        how="left"
    ).rename(columns={CANTON_COL: "canton"})

    # 5) canton-year (media de estaciones)
    df_canton = df_station.groupby(["canton", "year"], as_index=False)[["HDD", "CDD"]].mean()

    # 6) export
    df_station.to_csv(OUT_STATION, index=False)
    df_canton.to_csv(OUT_CANTON, index=False)

    print("Exportados:")
    print(" -", OUT_STATION)
    print(" -", OUT_CANTON)
    print(f"Archivos ignorados por formato: {skipped_files} | Archivos ausentes: {missing_files}")

    return df_station, df_canton

# ===============================
# Uso
# ===============================
df_hdd_station, df_hdd_canton = ensure_hdd_cdd(force_rebuild=False)
df_hdd_canton.head()



Columna cantón detectada: station_canton
Estaciones objetivo con tre200d0: 150
RAW_DIR usado: data\meteo\smn_daily_historical_by_station
Descarga faltantes -> OK: 0 | SKIP (ya existían): 150 | FAIL: 0


C:\Users\castrogf\AppData\Local\Temp\ipykernel_13040\1890660817.py:130: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["date"] = pd.to_datetime(df[DATE_COL], errors="coerce")
C:\Users\castrogf\AppData\Local\Temp\ipykernel_13040\1890660817.py:130: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["date"] = pd.to_datetime(df[DATE_COL], errors="coerce")
C:\Users\castrogf\AppData\Local\Temp\ipykernel_13040\1890660817.py:130: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["date"] = pd.to_datetime(df[DATE_COL], errors="coerce")
C:\Users\castrogf\AppData\Local\Temp\ipykernel_13040\1890660817.py:130: UserWarning: Parsing dates i

Exportados:
 - data\meteo\processed\smn_hdd_cdd_station_year_2000_2024.csv
 - data\meteo\processed\smn_hdd_cdd_canton_year_2000_2024.csv
Archivos ignorados por formato: 0 | Archivos ausentes: 0


,canton,year,HDD,CDD
0,AG,1984,1488.800000,5.200000
1,AG,1985,3590.400000,5.600000
2,AG,1986,3481.800000,9.800000
3,AG,1987,2442.766667,2.700000
4,AG,1988,2330.666667,3.633333
